In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import csv
from math import log10

In [ ]:
data = pd.read_csv('train.csv')

In [2]:
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text: str) -> str:
    return ' '.join([x for x in text.split() if x and x not in STOPWORDS])

not_word = [',', ':', '.', '?', '!', ')', '(', ';', '/', '-', '"', '\'', '_']

def cleaning(text:str) -> str:

  text = text.lower()
  text = remove_stopwords(text)

  for char in not_word:
    text = text.replace(char, ' ')

  return text


def count_words(l : list) -> dict:
  words = l.split()

  return Counter(words)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
class_grouped = data.groupby(by='discourse_type')

In [5]:
for key, gp in class_grouped:
  group = class_grouped.get_group(key)['discourse_text']
  l=''
  for i in range(len(group)):
    text = group.iloc[i]
    text = cleaning(text)
    l = l + text
  
  words_count = count_words(l)

  f = open(key + ".csv", "w")
  writer = csv.writer(f)
  for key, value in words_count.items():
    writer.writerow([key, value])

  f.close()



In [6]:

reader = csv.reader(open('Claim.csv', 'r'))
dict_claim = {}
for row in reader:
   k, v = row
   dict_claim[k] = v

reader = csv.reader(open('Concluding Statement.csv', 'r'))
dict_concluding = {}
for row in reader:
   k, v = row
   dict_concluding[k] = v

reader = csv.reader(open('Counterclaim.csv', 'r'))
dict_counterclaim = {}
for row in reader:
   k, v = row
   dict_counterclaim[k] = v


reader = csv.reader(open('Evidence.csv', 'r'))
dict_evidence = {}
for row in reader:
   k, v = row
   dict_evidence[k] = v

reader = csv.reader(open('Lead.csv', 'r'))
dict_lead = {}
for row in reader:
   k, v = row
   dict_lead[k] = v

reader = csv.reader(open('Position.csv', 'r'))
dict_position = {}
for row in reader:
   k, v = row
   dict_position[k] = v

reader = csv.reader(open('Rebuttal.csv', 'r'))
dict_rebuttal = {}
for row in reader:
   k, v = row
   dict_rebuttal[k] = v

In [47]:
import numpy as np

def correlation(word : str) -> np.array:

  n_claim = int(dict_claim[word]) if word in dict_claim else 0
  n_concluding = int(dict_concluding[word]) if word in dict_concluding else 0
  n_counterclaim = int(dict_counterclaim[word]) if word in dict_counterclaim else 0
  n_evidence = int(dict_evidence[word]) if word in dict_evidence else 0
  n_lead = int(dict_lead[word]) if word in dict_lead else 0
  n_position = int(dict_position[word]) if word in dict_position else 0
  n_rebuttal = int(dict_rebuttal[word]) if word in dict_rebuttal else 0


  total = n_claim + n_concluding + n_counterclaim + n_evidence + n_lead + n_position + n_rebuttal
  idf = log10(144293/total)
  return np.array((n_claim/len(dict_claim), n_concluding/len(dict_concluding), n_counterclaim/len(dict_counterclaim), \
                   n_evidence/(len(dict_evidence)), n_lead/len(dict_lead), n_position/len(dict_position), n_rebuttal/len(dict_rebuttal)))*idf


In [8]:
word_list = set()

for w in dict_claim:
  word_list.add(w)

for w in dict_concluding:
  word_list.add(w)

for w in dict_counterclaim:
  word_list.add(w)

for w in dict_evidence:
  word_list.add(w)

for w in dict_lead:
  word_list.add(w)

for w in dict_position:
  word_list.add(w)

for w in dict_rebuttal:
  word_list.add(w)




In [48]:
matrix = np.zeros((len(word_list), 7))
i = 0
for w in word_list:
  matrix[i, :] = correlation(w)
  i += 1
  

In [49]:
weights_matrix = pd.DataFrame(matrix, columns =['Claim', 'Concluding Statement', 'Counterclaim', 'Evidence', 'Lead', 'Position', 'Rebuttal'])
words = pd.DataFrame(word_list, columns=['Words'])
p = pd.concat([words, weights_matrix], axis=1)

In [53]:
p.to_csv('weights_matrix.csv')